## Task A

### 1. Finding Q

Covariance $Q$ characterizes the observation noise. In our case, observation is the vector:

$z = \begin{bmatrix} bearing \\ ID \end{bmatrix}$

Therefore, as variables in observation are not correlated:

$Q = \begin{bmatrix} \sigma_{bearing}^2 & 0 \\ 0 & \sigma_{ID}^2 \end{bmatrix}$

We know that observation of landmark ID has error equal to zero, and for bearing:

$\sigma_{bearing} = 0.35$

So we can reduce the covariance matrix to:

$Q = \begin{bmatrix} 0.35^2 \end{bmatrix}$

### 2. Finding Jacobians

We have the following odometry model:

$x = \begin{bmatrix} x \\ y \\ \theta \end{bmatrix}$

$u = \begin{bmatrix} \delta_{rot1} \\ \delta_{trans} \\ \delta_{rot2} \end{bmatrix}$

$x_t = g(x_{t-1}, u_t, \varepsilon_t) |_{\varepsilon_t=0} = \begin{bmatrix} x_{t-1} + \delta_{trans} cos(\theta + \delta_{rot1}) \\ y_{t-1} + \delta_{trans} sin(\theta + \delta_{rot1}) \\ \theta_{t-1} + \delta_{rot1} + \delta_{rot2}\end{bmatrix}$


According to it we can find state and control jacobians:

$G_t = \frac{\partial g(x_{t-1}, u_t, \varepsilon_t)}{\partial x_{t-1}} |_{\mu_{t-1}, \varepsilon_t=0} = \begin{bmatrix} 1 & 0 & -\delta_{trans} sin(\theta + \delta_{rot1}) \\ 0 & 1 & \delta_{trans} cos(\theta + \delta_{rot1}) \\ 0 & 0 & 1\end{bmatrix}$

$V_t = \frac{\partial g(x_{t-1}, u_t, \varepsilon_t)}{\partial u_{t}} |_{\mu_{t-1}, \varepsilon_t=0} = \begin{bmatrix} -\delta_{trans} sin(\theta + \delta_{rot1}) & cos(\theta + \delta_{rot1}) & 0 \\ \delta_{trans} cos(\theta + \delta_{rot1}) & sin(\theta + \delta_{rot1}) & 0 \\ 1 & 0 & 1\end{bmatrix}$

Our observations can be converted to features of landmarks:

$landmark_i = \begin{bmatrix} m_{i,x} & m_{i,y}\end{bmatrix}^T$

$f_i = \begin{bmatrix} range \\ bearing \\ signature \end{bmatrix} = \begin{bmatrix} r_{i,t} \\ \phi_{i,t} \\ s_{i,t} \end{bmatrix} = \begin{bmatrix} \sqrt{(m_{i,x}-x_t)^2 + (m_{i,y}-y_t)^2} \\ atan2(m_{i,x}-x_t, m_{i,y}-y_t) - \theta \\ 0 \end{bmatrix}$

and observation jacobian is:

$H_t = \frac{\partial g(x_{t-1}, x_t, \varepsilon_t)}{\partial x_{t}} |_{\mu_{t-1}, \varepsilon_t=0} = \begin{bmatrix} -\frac{m_{i,x} - \overline{\mu}_{t,x}}{r_{i,t}} & \frac{m_{i,y} - \overline{\mu}_{t,y}}{r_{i,t}} & 0 \\ \frac{m_{i,x} - \overline{\mu}_{t,x}}{r_{i,t}^2} & -\frac{m_{i,y} - \overline{\mu}_{t,y}}{r_{i,t}^2} & -1 \\ 0 & 0 & 0 \end{bmatrix}$

Reducing to bearing only:

$H_t = \begin{bmatrix} \frac{m_{i,x} - \overline{\mu}_{t,x}}{r_{i,t}^2} & -\frac{m_{i,y} - \overline{\mu}_{t,y}}{r_{i,t}^2} & -1 \end{bmatrix}$

Given the initial mean state and the initial control input:

$\mu_1 = \begin{bmatrix} 180 & 50 & 5 \end{bmatrix}^T$

$u = \begin{bmatrix} 0 & 10 & 0 \end{bmatrix}^T$

we obtain the following values of jacobians:

$G^{\mu_1}_t = \begin{bmatrix} 1 & 0 & 0 \\ 0 & 1 & 10 \\ 0 & 0 & 1 \end{bmatrix}$

$V^{\mu_1}_t = \begin{bmatrix} 0 & 1 & 0 \\ 10 & 0 & 0 \\ 1 & 0 & 1 \end{bmatrix}$

### 3. Finding R

Covariance $R$ characterizes the noise added to transition function and can be found via following equation:

$R_t = V_t M V_t^T$

where M is covariance of control noise and is found as following:

$M = \begin{bmatrix} \alpha_1 \delta_{rot1}^2 + \alpha_2 \delta_{trans}^2 & 0 & 0 \\ 0 & \alpha_3 \delta_{trans}^2 + \alpha_4 (\delta_{rot1}^2 + \delta_{rot2}^2) & 0 \\ 0 & 0 & \alpha_1 \delta_{rot2}^2 + \alpha_2 \delta_{trans}^2 \end{bmatrix}$

$\alpha_1 = 0.05^2$

$\alpha_2 = 0.001^2$

$\alpha_3 = 0.05^2$

$\alpha_4 = 0.01^2$